### Monta Tabela de Anotação

In [42]:
import collections
import re
import pandas as pd

dictF = collections.OrderedDict()
anotador = 0
cont = 1
str1 = ""

for filename in ['Clear_01_1_ISO_Andressa_Bressani.xml','Clear_01_1_ISO_Eric_Eltz.xml', 'Clear_01_1_ISO_Keli_Schell.xml' ]:
    
    with open(filename,'r') as file:
        cont = 1
        for line in file:

            if line.find("EM-ID") >= 0 and line.find("/EM") >= 0:
                str1 = re.findall('(?<=>).*?(?=<)', line)[0] + " - " + str(cont)
                categoria = re.findall('(?<=CATEG=").*?(?=")', line)[0]
                #print(str1)
                if str1 in dictF:
                    dictF[str1][anotador] = categoria
                else:
                    dictF[str1] = ['','','']
                    dictF[str1][anotador] = categoria
            
            elif line.find("EM-ID") >= 0:
                str1 = re.findall('(?<=>).*?(?=$)', line)[0]
                str1 = str1.replace("\n", "")
                categoria = re.findall('(?<=CATEG=").*?(?=")', line)[0]
            
            elif line.find("/EM") >= 0:
                str1 = str1 + " " + re.findall('(?<=^).*?(?=<)', line)[0] + " - " + str(cont)
                str1 = str1.replace("\n", "")
                if str1 in dictF:
                    dictF[str1][anotador] = categoria
                else:
                    dictF[str1] = ['','','']
                    dictF[str1][anotador] = categoria
                str1 = ""
            
            else:
                str1 = str1 + " " + line
                
            cont += 1
        
    anotador += 1
    
print(len(dictF))
    
kappa_pd = pd.DataFrame.from_dict(dictF,orient='index')
kappa_pd[:10]

286


,0,1,2
zonas de cisalhamento - 6,CONTEXTOgeologicoDeBACIA,,CONTEXTOgeologicoDeBACIA
neoproterozoicas - 41,era,eon,era
Bacia do Paraná - 54,baciaSEDIMENTAR,baciaSEDIMENTAR,baciaSEDIMENTAR
Fanerozoico - 58,EON,EON,EON
Zona de Cisalhamento - 99,CONTEXTOgeologicoDeBACIA,,CONTEXTOgeologicoDeBACIA
zonas de cisalhamento - 155,CONTEXTOgeologicoDeBACIA,,CONTEXTOgeologicoDeBACIA
Bacia do Paraná - 168,baciaSEDIMENTAR,baciaSEDIMENTAR,baciaSEDIMENTAR
Zona de Cisalhamento - 174,CONTEXTOgeologicoDeBACIA,,CONTEXTOgeologicoDeBACIA
Zona de Cisalhamento - 294,CONTEXTOgeologicoDeBACIA,,CONTEXTOgeologicoDeBACIA
fanerozoicas - 326,eon,eon,eon


### Montagem da Tabela para o Kappa

In [43]:
classes = set()

for x in kappa_pd[0].unique():
    classes.add(x.lower())
    
for x in kappa_pd[1].unique():
    classes.add(x.lower())
    
for x in kappa_pd[2].unique():
    classes.add(x.lower())

class_id = 0
for clas in classes:
    print(str(class_id) + clas)
    class_id += 1

0
1unidadeestratigrafica
2formacao
3idade
4contextogeologicodebacia
5outro
6baciasedimentar
7periodo
8era
9eon


In [44]:
w, h = len(classes), len(kappa_pd);
kappa_table = [[0 for x in range(w)] for y in range(h)] 

row_id = 0
for index, row in kappa_pd.iterrows():
    class_id = 0
    #print(row)
    for clas in classes:
        #print(str(class_id) + clas)
        if row[0].lower() == clas:
            kappa_table[row_id][class_id] += 1
        if row[1].lower() == clas:
            kappa_table[row_id][class_id] += 1
        if row[2].lower() == clas:
            kappa_table[row_id][class_id] += 1
        class_id += 1
    row_id += 1
    #break
            
kappa_table[:10] #sample 10

[[1, 0, 0, 0, 2, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 2, 1],
 [0, 0, 0, 0, 0, 0, 3, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 3],
 [1, 0, 0, 0, 2, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 2, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 3, 0, 0, 0],
 [1, 0, 0, 0, 2, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 2, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 3]]

In [23]:
from statsmodels.stats.inter_rater import fleiss_kappa
fleiss_kappa(kappa_table)

0.15955482017731867

### Cálculo do Fleiss Kappa manualmente

In [9]:
NC = 3 * len(kappa_table)

Z = 0 
for vec in kappa_table:
    S = 0
    for clas in vec:
        S += (clas * (clas - 1))
    S = (1 / (3 * (3 - 1))) * S
    if Z < 10: # sample
        print(vec)
        print(S)
    Z += S

[1, 0, 0, 0, 2, 0, 0, 0, 0, 0]
0.3333333333333333
[0, 0, 0, 0, 0, 0, 0, 0, 2, 1]
0.3333333333333333
[0, 0, 0, 0, 0, 0, 3, 0, 0, 0]
1.0
[0, 0, 0, 0, 0, 0, 0, 0, 0, 3]
1.0
[1, 0, 0, 0, 2, 0, 0, 0, 0, 0]
0.3333333333333333
[1, 0, 0, 0, 2, 0, 0, 0, 0, 0]
0.3333333333333333
[0, 0, 0, 0, 0, 0, 3, 0, 0, 0]
1.0
[1, 0, 0, 0, 2, 0, 0, 0, 0, 0]
0.3333333333333333
[1, 0, 0, 0, 2, 0, 0, 0, 0, 0]
0.3333333333333333
[0, 0, 0, 0, 0, 0, 0, 0, 0, 3]
1.0
[0, 0, 0, 0, 0, 0, 3, 0, 0, 0]
1.0
[0, 0, 0, 0, 0, 0, 0, 0, 0, 3]
1.0
[0, 0, 0, 0, 0, 0, 3, 0, 0, 0]
1.0
[0, 0, 0, 0, 0, 0, 0, 0, 0, 3]
1.0


In [10]:
PA = Z / len(kappa_table)
PA

0.43006993006992733

In [11]:
NC = 3 * len(kappa_table)
NC

858

In [12]:
class_sum = pd.DataFrame(data=kappa_table).sum(axis=0)
class_sum

0    417
1     30
2      1
3      2
4    233
5     42
6     61
7      5
8     26
9     41
dtype: int64

In [13]:
PE = 0
for clas in class_sum:
    print(str(clas) + "/" +str(NC))
    PE += (clas/NC) * (clas/NC)
PE

417/858
30/858
1/858
2/858
233/858
42/858
61/858
5/858
26/858
41/858


0.32187121347960507

In [14]:
K = (PA - PE) / (1 - PE)
K

0.15955482017731473

### Calculo Kappa de Identificação

In [28]:
w, h = 2, len(kappa_pd);
kappa_id_table = [[0 for x in range(w)] for y in range(h)]

row_id = 0
for index, row in kappa_pd.iterrows():
    class_id = 0
    for clas in classes:
        if row[0].lower() == clas:
            kappa_id_table[row_id][class_id] += 1
        if row[1].lower() == clas:
            kappa_id_table[row_id][class_id] += 1
        if row[2].lower() == clas:
            kappa_id_table[row_id][class_id] += 1
        if class_id == 0:
            class_id += 1
    row_id += 1
    
kappa_id_table[:10]

[[1, 2],
 [0, 3],
 [0, 3],
 [0, 3],
 [1, 2],
 [1, 2],
 [0, 3],
 [1, 2],
 [1, 2],
 [0, 3]]

In [24]:
fleiss_kappa(kappa_id_table)

-0.10109463449648438

### Calculo Kappa da Classificação

In [48]:
kappa_class_table = pd.DataFrame(data=kappa_table)

print(kappa_class_table.shape)

class_id = 0
for clas in classes:
    print("")
    print(str(class_id) + clas)
    kappa_class_table = kappa_class_table[kappa_class_table[class_id]==0]
    kappa_class_table.drop(kappa_class_table.columns[class_id],inplace=True,axis=1)
    print(kappa_class_table.shape)
    print(fleiss_kappa(kappa_class_table.as_matrix()))
    kappa_class_table = pd.DataFrame(data=kappa_table)
    class_id += 1

(286, 10)

0
(50, 9)
0.891120251633

1unidadeestratigrafica
(261, 9)
0.15875780816

2formacao
(285, 9)
0.157442856218

3idade
(284, 9)
0.160596248461

4contextogeologicodebacia
(110, 9)
0.436501954873

5outro
(245, 9)
0.187080785571

6baciasedimentar
(265, 9)
0.0331993207811

7periodo
(284, 9)
0.152891313402

8era
(275, 9)
0.118795440266

9eon
(268, 9)
0.077834400453
